In [ ]:
### Imports

import numpy as np
from astropy.io import fits
from astropy.stats import sigma_clipped_stats
from scipy.ndimage import label, maximum_position, distance_transform_edt
import matplotlib.pyplot as plt
import os
import csv

## Helper Function

In [ ]:
### Function to filter through potential sources and assess whether the candidate is a local maximum and has a sufficient radial profile

def is_star_candidate(image, x, y, X_full, Y_full, box_size=7, inner_radius=3, outer_radius=7, min_contrast=2.0):

    x_int = int(round(x))
    y_int = int(round(y))
    
    half_box = box_size // 2
    y_min = max(0, y_int - half_box)
    y_max = min(image.shape[0], y_int + half_box + 1)
    x_min = max(0, x_int - half_box)
    x_max = min(image.shape[1], x_int + half_box + 1)
    sub_box = image[y_min:y_max, x_min:x_max]
    if np.nanmax(sub_box) != image[y_int, x_int]:
        return False

    local_y_min = max(0, y_int - outer_radius)
    local_y_max = min(image.shape[0], y_int + outer_radius + 1)
    local_x_min = max(0, x_int - outer_radius)
    local_x_max = min(image.shape[1], x_int + outer_radius + 1)
    Y_local = Y_full[local_y_min:local_y_max, local_x_min:local_x_max]
    X_local = X_full[local_y_min:local_y_max, local_x_min:local_x_max]
    
    r = np.sqrt((X_local - x_int)**2 + (Y_local - y_int)**2)
    annulus_mask = (r >= inner_radius) & (r < outer_radius)
    if np.sum(annulus_mask) < 2:
        return False
    annulus_median = np.nanmedian(image[local_y_min:local_y_max, local_x_min:local_x_max][annulus_mask])
    annulus_std = np.nanstd(image[local_y_min:local_y_max, local_x_min:local_x_max][annulus_mask])
    
    return image[y_int, x_int] > annulus_median + min_contrast * annulus_std

## Main Source Subtraction Function

In [ ]:
def psf_subtraction_cycle(detection_image, working_image, cycle_label, psf_data, output_dir,
                          mosaic_header, permanent_mask, mask_distance, min_mask_sep=3,
                          catalog_filename=None, threshold_sigma=3.0):

    print(f"\n--- Starting Cycle {cycle_label} ---")
    
    # Calculate Background Statistics
    _, global_median, global_std = sigma_clipped_stats(detection_image, sigma=3.0)
    threshold = global_median + threshold_sigma * global_std
    print(f"Cycle {cycle_label}: Global median = {global_median:.3f}, std = {global_std:.3f}, threshold = {threshold:.3f}")
    
    # Identify Potential Sources and Positions
    bmask = detection_image > threshold
    labeled, num_regions = label(bmask)
    print(f"Cycle {cycle_label}: Number of connected regions above threshold: {num_regions}")
    
    initial_positions = maximum_position(detection_image, labels=labeled,
                                          index=np.arange(1, num_regions+1))

    # Filter Potential Sources and Create Final Source List
    source_list = []
    Y_full, X_full = np.indices(detection_image.shape)
    for pos in initial_positions:
        y, x = pos
        
        if mask_distance[y, x] < min_mask_sep:
            continue
            
        brightness = detection_image[y, x]
        if np.isnan(brightness):
            continue
            
        if is_star_candidate(detection_image, x, y, X_full, Y_full, box_size=1, inner_radius=1, outer_radius=2, min_contrast=0.5):
            source_list.append((x, y, brightness))

    # Sort Final List by Decreasing Brightness
    source_list = sorted(source_list, key=lambda s: s[2], reverse=True)
    
    print(f"Cycle {cycle_label}: {len(source_list)} candidate sources passed.")

    # Initialize List to Keep Track of Subtracted Sources
    new_subtracted_sources = []
    
    # Define Magnitude Zero-Point
    mag_zeropoint = 20.787  # Set by user based on mosaic in use

    # Establish Header for Catalog - can be set by user
    if catalog_filename is not None and not os.path.exists(catalog_filename):
        with open(catalog_filename, "w") as f:
            f.write("# Cycle  SourceIndex   X(pix)   Y(pix)   PeakDN   ScalingFactor   ABmag\n")
            f.write("# -----------------------------------------------------------------------\n")
    
    # Iterate Over Each Source
    for i, (x, y, bright) in enumerate(source_list, start=1):
        
        psf_shape = psf_data.shape
        half_psf_x = psf_shape[1] // 2
        half_psf_y = psf_shape[0] // 2
        x_int, y_int = int(round(x)), int(round(y))
        xmin = max(0, x_int - half_psf_x)
        xmax = min(working_image.shape[1], x_int + half_psf_x)
        ymin = max(0, y_int - half_psf_y)
        ymax = min(working_image.shape[0], y_int + half_psf_y)
        
        # Extract Region Corresponding to PSF Model
        region = working_image[ymin:ymax, xmin:xmax]
        psf_resized = psf_data[:region.shape[0], :region.shape[1]]
        
        # Calculate PSF Scaling Factor
        s = np.nansum(region * psf_resized) / np.nansum(psf_resized**2)
        
        # Compute AB Magnitude 
        if bright > 0:
            peak_mag = -2.5 * np.log10(s) + mag_zeropoint
        else:
            # Dummy value to save if brightness is not a positive value
            peak_mag = 99.999
        
        # Save Statistics to Catalog
        if catalog_filename is not None:
            with open(catalog_filename, "a") as f:
                f.write(f"{cycle_label:<5d} {i:<12d} {x:9.1f} {y:8.1f} {bright:9.2f} {s:14.3f} {peak_mag:9.3f}\n")
        
        # PSF Subtraction
        working_image[ymin:ymax, xmin:xmax] -= s * psf_resized

        # Save Subtracted Source to List
        new_subtracted_sources.append((x, y))
    
    # Save Updated Residual Image After Each Iteration
    out_file = os.path.join(output_dir, f'final_psf_subtracted_cycle{cycle_label}.fits')
    final_display = np.nan_to_num(working_image, nan=0)
    fits.PrimaryHDU(final_display, header=mosaic_header).writeto(out_file, overwrite=True)
    print(f"Cycle {cycle_label}: Saved PSF-subtracted image as {out_file}")
    
    # Write Total Number of Sources Subtracted in this Cycle to Catalog
    if catalog_filename is not None:
        with open(catalog_filename, "a") as f:
            f.write(f"# Cycle {cycle_label}: {len(new_subtracted_sources)} sources subtracted\n")
    
    return working_image, new_subtracted_sources

## Validation Helper Functions

In [ ]:
### Function to Insert Artificial Sources into Mosaic for Testing

def inject_fake_sources(image, psf_data, n_sources=250, scaling_factors=[5, 10, 15, 20]):
    np.random.seed(None)
    injected_image = image.copy()
    injected_catalog = []
    psf_shape = psf_data.shape
    half_psf_x = psf_shape[1] // 2
    half_psf_y = psf_shape[0] // 2
    ny, nx = image.shape
    x_min, x_max = half_psf_x, nx - half_psf_x - 1
    y_min, y_max = half_psf_y, ny - half_psf_y - 1
    
    for scale_factor in scaling_factors:
        for i in range(n_sources):
            x = np.random.randint(x_min, x_max)
            y = np.random.randint(y_min, y_max)
            
            x_start = x - half_psf_x
            x_end = x_start + psf_shape[1]
            y_start = y - half_psf_y
            y_end = y_start + psf_shape[0]
            injected_image[y_start:y_end, x_start:x_end] += scale_factor * psf_data
            
            injected_catalog.append((x, y, scale_factor))
    return injected_image, injected_catalog

In [ ]:
### Function to Identify Matched and Unmatched Artificial Sources 

def match_catalogs(injected_catalog, detected_catalog, max_sep=5.0):
    matches = []
    unmatched_injected = []
    for inj in injected_catalog:
        inj_x, inj_y, inj_scale = inj
        best_dist = np.inf
        best_det = None
        for det in detected_catalog:
            det_x, det_y, det_val = det
            dist = np.sqrt((inj_x - det_x)**2 + (inj_y - det_y)**2)
            if dist < best_dist:
                best_dist = dist
                best_det = det
        if best_dist <= max_sep:
            matches.append((inj, best_det, best_dist))
        else:
            unmatched_injected.append(inj)
    return matches, unmatched_injected

In [ ]:
### Function to Measure and Plot Completeness, and Save Plot to Output Directory

def plot_completeness(matches, injected_catalog, brightness_bins, mosaic_type='4.5', output_dir=None):

    # mosaic type between channels; can be reset by user
    if mosaic_type == '3.6':
        zeropoint = 20.787
    elif mosaic_type == '4.5':
        zeropoint = 20.798
    else:
        raise ValueError("Invalid mosaic type. Choose '3.6' or '4.5'.")

    # scaling factors
    inj_scales = np.array([src[2] for src in injected_catalog])
    det_scales = np.array([det[2] for (_, det, _) in matches]) 

    counts_injected, bins = np.histogram(inj_scales, bins=brightness_bins)
    counts_matched, _ = np.histogram(det_scales, bins=brightness_bins)
    completeness = counts_matched / np.maximum(counts_injected, 1)

    bin_centers = 0.5 * (bins[1:] + bins[:-1])
    mag_bin_centers = -2.5 * np.log10(bin_centers) + zeropoint

    plt.figure(figsize=(8,6))
    plt.plot(mag_bin_centers, completeness, 'o-', label='Completeness')
    plt.xlabel('Injected Magnitude')
    plt.ylabel('Completeness')
    plt.title('Completeness vs. Injected Magnitude')
    plt.ylim(0,1.1)
    plt.grid(True)
    plt.legend()
    plt.gca().invert_xaxis()
    if output_dir is not None:
        plt.savefig(os.path.join(output_dir, "completeness_vs_magnitude.png"), bbox_inches='tight')
    plt.show()

In [ ]:
### Function to Measure and Plot Photometric Error, and Save Plot to Output Directory

def plot_photometric_error(matches, mosaic_type='4.5', output_dir=None):

    # mosaic type between channels; can be reset by user
    if mosaic_type == '3.6':
        zeropoint = 20.787
    elif mosaic_type == '4.5':
        zeropoint = 20.798
    else:
        raise ValueError("Invalid mosaic type. Choose '3.6' or '4.5'.")

    # scaling factors
    inj_scales = np.array([inj[2] for (inj, det, sep) in matches])
    fit_scales = np.array([det[2] for (inj, det, sep) in matches])

    true_m = -2.5 * np.log10(inj_scales) + zeropoint
    calc_m = -2.5 * np.log10(fit_scales) + zeropoint
    errors = (true_m - calc_m) / true_m

    plt.figure(figsize=(8,6))
    plt.plot(true_m, errors, 'o', label='Fractional Error')
    plt.xlabel('Magnitude')
    plt.ylabel('Photometric Error')
    plt.title('Photometric Accuracy vs. Test Source Magnitude')
    plt.axhline(0, color='gray', linestyle='--')
    plt.grid(True)
    if output_dir is not None:
        plt.savefig(os.path.join(output_dir, "photometric_accuracy_vs_magnitude.png"), bbox_inches='tight')
    plt.show()

## Main Iterative Workflow and Validation Routine

In [ ]:
# Paths to be Set by User
mosaic_file = 'mosaic_file'
psf_file = 'psf_file'
output_dir = 'output_directory'
os.makedirs(output_dir, exist_ok=True)

# Establish Catalog
catalog_filename = os.path.join(output_dir, "source_catalog.tex")

# Load Data
with fits.open(mosaic_file) as hdu:
    data = np.copy(hdu[0].data)
    mosaic_header = hdu[0].header

with fits.open(psf_file) as hdu_psf:
    psf_data = np.copy(hdu_psf[0].data)

# Print Mosaic Dimensions
ny, nx = data.shape
print(f"Mosaic dimensions: {nx} x {ny} pixels")

# Define Submosaic Cutout to Validate On
cut_size = 4000
if cut_size > nx or cut_size > ny:
    raise ValueError("cut_size is larger than the mosaic dimensions.")
center_x = nx // 2
center_y = ny // 2
half_cut = cut_size // 2
sub_mosaic = data[center_y-half_cut:center_y+half_cut, center_x-half_cut:center_x+half_cut]
print(f"Sub-mosaic dimensions: {sub_mosaic.shape}")

# Save Submosaic to Output Directory
sub_mosaic_file = os.path.join(output_dir, 'sub_mosaic.fits')
fits.PrimaryHDU(sub_mosaic, header=mosaic_header).writeto(sub_mosaic_file, overwrite=True)
print(f"Saved sub_mosaic as {sub_mosaic_file}")

### Routine to Mask Out Saturated Pixels

saturation_threshold = 60.0              # saturation threshold; to be adjusted by user
small_initial_mask_radius = 160          # small mask radius; to be adjusted by user  
large_initial_mask_radius = 220          # large mask radius; to be adjusted by user 

# Define Global Background
global_bg = np.nanmedian(sub_mosaic)

# Identify Saturated Regions
saturated_pixels = sub_mosaic > saturation_threshold
labeled_saturated, num_sat = label(saturated_pixels)
print(f"Number of saturated regions detected: {num_sat}")

# Iterate Through Saturated Regions and Collect Statistics
region_info = []
for region_label in range(1, num_sat+1):
    y_indices, x_indices = np.where(labeled_saturated == region_label)
    if len(y_indices) == 0:
        continue
    region_values = sub_mosaic[y_indices, x_indices]
    if np.nanmax(region_values) < min_flux_threshold:
        continue
    peak_index = np.argmax(region_values)
    y_peak, x_peak = y_indices[peak_index], x_indices[peak_index]
    distances_region = np.sqrt((x_indices - x_peak)**2 + (y_indices - y_peak)**2)
    effective_radius = np.max(distances_region)
    x_min_box, x_max_box = np.min(x_indices), np.max(x_indices)
    y_min_box, y_max_box = np.min(y_indices), np.max(y_indices)
    width = x_max_box - x_min_box + 1
    height = y_max_box - y_min_box + 1
    aspect_ratio = width / height if height != 0 else 1.0
    peak_flux = np.nanmax(region_values)
    region_info.append((region_label, y_peak, x_peak, effective_radius, aspect_ratio, peak_flux))

# Calculate Average Effective Radius of Saturated Regions
if region_info:
    avg_effective_radius = np.mean([info[3] for info in region_info])
else:
    avg_effective_radius = 0
print(f"Average effective radius = {avg_effective_radius:.1f} pixels")

# Set Mask Size for Each Region Based on Effective Radius
for (region_label, y_peak, x_peak, effective_radius, aspect_ratio, peak_flux) in region_info:
    if effective_radius <= avg_effective_radius * 1.5:
        r_final = small_initial_mask_radius
    else:
        r_final = large_initial_mask_radius
    Y, X = np.indices(sub_mosaic.shape)
    distance_arr = np.sqrt((X - x_peak)**2 + (Y - y_peak)**2)
    final_mask = distance_arr <= r_final
    sub_mosaic[final_mask] = np.nan

permanent_mask = np.isnan(sub_mosaic)

# Sets Mask Separation Distance - Only Sources At Least the 'min_mask_sep' Distance from Masked Regions Will be Considered
mask_distance = distance_transform_edt(~permanent_mask)
min_mask_sep = 2

# Make Copies of Sub-Mosaic and Masked Image; Save Masked Image to Output Directory
original_sub_mosaic = sub_mosaic.copy()
masked_image = original_sub_mosaic.copy()
masked_image[permanent_mask] = 0
masked_image_file = os.path.join(output_dir, "masked_image.fits")
fits.PrimaryHDU(masked_image, header=mosaic_header).writeto(masked_image_file, overwrite=True)
print(f"Masked image saved as {masked_image_file}")


### Add in Artificial Sources 

scaling_factors = [2, 6, 12, 24]  # scaling factors to be set by user

# Number of sources to be added (n_sources) to be set by user
test_image, injected_catalog = inject_fake_sources(sub_mosaic, psf_data, n_sources=300, scaling_factors=scaling_factors)
print(f"Injected {len(injected_catalog)} fake sources for validation.")

# Save Test Image to Output Directory
test_image_file = os.path.join(output_dir, "test_image_with_injected_sources.fits")
fits.PrimaryHDU(test_image, header=mosaic_header).writeto(test_image_file, overwrite=True)
print(f"Test image with fake sources saved as {test_image_file}")

# Get Number of Artificial Sources that Fall in Masked Regions
masked_counts = {val: 0 for val in scaling_factors}
nonmasked_counts = {val: 0 for val in scaling_factors}
psf_shape = psf_data.shape
half_psf_x = psf_shape[1] // 2
half_psf_y = psf_shape[0] // 2

for (x, y, scale_factor) in injected_catalog:
    x_start = x - half_psf_x
    x_end = x_start + psf_shape[1]
    y_start = y - half_psf_y
    y_end = y_start + psf_shape[0]
    if np.all(permanent_mask[y_start:y_end, x_start:x_end]):
        masked_counts[scale_factor] += 1
    else:
        nonmasked_counts[scale_factor] += 1

print("Artificial Sources Masked Counts:")
for val in scaling_factors:
    print(f"Scale {val}: Masked: {masked_counts[val]}, Non-masked: {nonmasked_counts[val]}")

with open(catalog_filename, "a") as f:
    f.write("%% Artificial Sources Masked Counts:\n")
    for val in scaling_factors:
        f.write(f"Scaling {val}: Masked: {masked_counts[val]}, Non-masked: {nonmasked_counts[val]} \\\\\n")

# Display Image with Marked Artificial Sources Overlaid
plt.figure(figsize=(8,8))
plt.imshow(test_image, origin='lower', cmap='gray')
color_map = {2: 'blue', 6: 'green', 12: 'orange', 24: 'red'}  # should match scaling_factors

for val in sorted(color_map.keys()):
    xs = [x for (x, y, s) in injected_catalog if s == val]
    ys = [y for (x, y, s) in injected_catalog if s == val]
    plt.plot(xs, ys, marker='x', linestyle='None', color=color_map[val],
             markersize=10, markeredgewidth=2, label=f'Scale {val}')
plt.title("Test Image with Injected Fake Sources Marked")
plt.colorbar()
plt.legend()
marked_file = os.path.join(output_dir, "test_image_with_injected_sources_marked.png")
plt.savefig(marked_file, bbox_inches='tight')
plt.show()

# Save Catalog of Artificial Sources to CSV
catalog_file = os.path.join(output_dir, "injected_sources_catalog.csv")
with open(catalog_file, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["X", "Y", "ScalingFactor"])
    for x, y, scale_factor in injected_catalog:
        writer.writerow([x, y, scale_factor])
print("Injected sources catalog saved as", catalog_file)

# Define Bins for the Magnitudes
brightness_bins = np.array([1, 4, 10, 16, 26])
brightness_keys = [2, 6, 12, 24]
completeness_by_bin = {k: [] for k in brightness_keys}
normalized_completeness_by_bin = {k: [] for k in brightness_keys}
photometric_error_by_bin = {k: [] for k in brightness_keys}
detection_counts = []

### Iterative Cycle Set Up

ncycles = 10   # number of cycles: set by user (should be higher to ensure proper validation)

cumulative_working = test_image.copy()
cumulative_exclusion_mask = np.zeros_like(test_image, dtype=bool)
all_new_sources = []

zeropoint = 20.798  # zero point value to be set by user

# Iterate Through All Cycles
for cycle in range(1, ncycles+1):
    print(f"\n========== Starting Iterative Cycle {cycle} ==========")
    detection_image = cumulative_working.copy()
    detection_image[cumulative_exclusion_mask] = np.nan
    
    if cycle == 1:
        tsig = 5.0      # threshold above background; set by user
    else:
        tsig = 3.0      # optional; lower threshold after first iteration

    # PSF Subtraction Function
    cumulative_working, new_sources = psf_subtraction_cycle(
        detection_image, cumulative_working, cycle, psf_data, output_dir,
        mosaic_header, permanent_mask, mask_distance, min_mask_sep,
        catalog_filename, threshold_sigma=tsig)
    all_new_sources.extend(new_sources)
    
    # Update Exclusion Mask; Prevents Re-Subtraction
    box_half = 1 # set size for each source
    for (x, y, s) in new_sources:
        x_center = int(round(x))
        y_center = int(round(y))
        x_min_box = max(0, x_center - box_half)
        x_max_box = min(test_image.shape[1], x_center + box_half + 1)
        y_min_box = max(0, y_center - box_half)
        y_max_box = min(test_image.shape[0], y_center + box_half + 1)
        box = cumulative_working[y_min_box:y_max_box, x_min_box:x_max_box]
        if np.all(np.isnan(box)):
            continue
        flat = box.flatten()
        valid = np.where(~np.isnan(flat))[0]
        if len(valid) == 0:
            continue
        sorted_idx = valid[np.argsort(flat[valid])[::-1]]
        for idx in sorted_idx[:3]:
            row = y_min_box + idx // (x_max_box - x_min_box)
            col = x_min_box + idx % (x_max_box - x_min_box)
            cumulative_exclusion_mask[row, col] = True

    detected_catalog_cycle = all_new_sources.copy()
    detection_counts.append(len(detected_catalog_cycle))
    
    # Match Detected Sources with Artificial Sources to Determine Which Artificial Sources Were Identified
    matches_cycle, unmatched_cycle = match_catalogs(injected_catalog, detected_catalog_cycle, max_sep=5.0)
    purity_cycle = len(matches_cycle) / len(detected_catalog_cycle) if len(detected_catalog_cycle) > 0 else 0
    print(f"Cycle {cycle} Metrics:")
    print(f"  Cumulative fitted sources: {len(detected_catalog_cycle)}")
    print(f"  Matched sources: {len(matches_cycle)}")
    print(f"  Purity: {purity_cycle:.2f}")
    
    # Calculate Completeness per Magnitude Bin
    for i, (low, high) in enumerate(zip(brightness_bins[:-1], brightness_bins[1:])):
        key = brightness_keys[i]
        inj_in_bin = [src for src in injected_catalog if low <= src[2] < high]
        n_injected = len(inj_in_bin)
        matched_in_bin = [m for m in matches_cycle if low <= m[0][2] < high]
        n_matched = len(matched_in_bin)
        comp = n_matched / n_injected if n_injected > 0 else np.nan
        completeness_by_bin[key].append(comp)
        
        # Normalized Completeness - Normalizes over all Non-Masked Artificial Sources (Since Those Over Masks Cannot be Detected)
        norm_comp = n_matched / nonmasked_counts[key] if nonmasked_counts[key] > 0 else np.nan
        normalized_completeness_by_bin[key].append(norm_comp)
    
        # Calculate Photometric Error per Magnitude Bin
        errors = []
        for match in matched_in_bin:
            inj_scale = match[0][2]
            fit_scale = match[1][2]
            # Magnitude from scaling_factor
            true_m = -2.5 * np.log10(inj_scale) + zeropoint
            calc_m = -2.5 * np.log10(fit_scale) + zeropoint
            error = (true_m - calc_m) / true_m
            errors.append(error)
        avg_error = np.mean(errors) if errors else np.nan
        photometric_error_by_bin[key].append(avg_error)

    # Save Statistics
    metrics_file = os.path.join(output_dir, f"metrics_cycle{cycle}.txt")
    with open(metrics_file, "w") as mf:
        mf.write(f"Cycle {cycle} Metrics:\n")
        mf.write(f"  Cumulative fitted sources: {len(detected_catalog_cycle)}\n")
        mf.write(f"  Matched sources: {len(matches_cycle)}\n")
    print(f"Cycle {cycle} metrics saved as {metrics_file}")

# Plot Cumulative Detection Counts and Save Plot
cycles = np.arange(1, ncycles+1)
plt.figure(figsize=(8,6))
plt.plot(cycles, detection_counts, 'o-', label='Cumulative Fitted Sources per Iteration')
plt.xlabel('Iteration Number')
plt.ylabel('Number of Fitted Sources')
plt.title('Cumulative Detection Count per Iteration')
plt.grid(True)
plt.legend()
cumulative_detection_file = os.path.join(output_dir, "cumulative_detection_count.png")
plt.savefig(cumulative_detection_file, bbox_inches='tight')
plt.show()
print("Cumulative detection count plot saved as", cumulative_detection_file)

# Plot Completeness vs. Iteration Per Bin and Save Plot
plt.figure(figsize=(10,6))
for key in brightness_keys:
    mag_value = -2.5 * np.log10(key) + zeropoint
    plt.plot(cycles, completeness_by_bin[key], marker='o', label=f'Mag {mag_value:.2f}')
plt.xlabel('Iteration Number')
plt.ylabel('Cumulative Completeness')
plt.title('Cumulative Completeness vs. Iteration (Scaling Factor Bins)')
plt.ylim(0,1.1)
plt.legend(title='Magnitude')
plt.grid(True)
comp_cycle_file = os.path.join(output_dir, "completeness_vs_cycle_by_flux.png")
plt.savefig(comp_cycle_file, bbox_inches='tight')
plt.show()
print("Per-bin completeness plot saved as", comp_cycle_file)

# Plot Normalized Completeness vs. Iteration and Save Plot
plt.figure(figsize=(10,6))
for key in brightness_keys:
    mag_value = -2.5 * np.log10(key) + zeropoint
    plt.plot(cycles, normalized_completeness_by_bin[key], marker='o', label=f'Mag {mag_value:.2f}')
plt.xlabel('Iteration Number')
plt.ylabel('Normalized Completeness')
plt.title('Normalized Completeness vs. Iteration for Each Magnitude Bin')
plt.legend(title='Magnitude')
plt.grid(True)
norm_comp_cycle_file = os.path.join(output_dir, "normalized_completeness_vs_cycle.png")
plt.savefig(norm_comp_cycle_file, bbox_inches='tight')
plt.show()
print("Normalized completeness plot saved as", norm_comp_cycle_file)

# Plot Photometric Error vs. Iteration and Save Plot
plt.figure(figsize=(10,6))
for key in brightness_keys:
    mag_value = -2.5 * np.log10(key) + zeropoint
    plt.plot(cycles, photometric_error_by_bin[key], marker='o', label=f'Mag {mag_value:.2f}')
plt.xlabel('Iteration Number')
plt.ylabel('Average Fractional Photometric Error')
plt.title('Average Photometric Error vs. Iteration per Magnitude Bin')
plt.legend(title='Magnitude')
plt.grid(True)
phot_cycle_file = os.path.join(output_dir, "photometric_error_vs_cycle_by_flux.png")
plt.savefig(phot_cycle_file, bbox_inches='tight')
plt.show()
print("Per-bin photometric error plot saved as", phot_cycle_file)

# Print Final Matched Statistics
print(f"Overall: Total cumulative fitted sources: {len(all_new_sources)}")
matches_overall, unmatched_overall = match_catalogs(injected_catalog, all_new_sources, max_sep=5.0)
print(f"Number of artificial sources: {len(injected_catalog)}")
print(f"Number of matched sources: {len(matches_overall)}")
print(f"Number of missed artificial sources: {len(unmatched_overall)}")

print("Final counts per magnitude:")
for val in brightness_keys:
    total_injected = sum(1 for src in injected_catalog if src[2] == val)
    total_matched = sum(1 for (inj, det, sep) in matches_overall if inj[2] == val)
    total_unmatched = total_injected - total_matched
    print(f"Scale {val}: Artificial: {total_injected}, Matched: {total_matched}, Unmatched: {total_unmatched}")

plot_completeness(matches_overall, injected_catalog, brightness_bins=brightness_bins, mosaic_type='4.5', output_dir=output_dir)
plot_photometric_error(matches_overall, mosaic_type='4.5', output_dir=output_dir)

The above Source Subtraction and Source Counts Validation Code was developed by Emily McCallum as part of her Applied Mathematics Senior Thesis at Harvard College. Latest update: 27 Mar 2025